In [101]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
dataset = pd.read_csv('NVDA_Revenue_Prediction_Data.csv')
dataset.dropna(axis=1,how='all',inplace=True)
dataset

,year-quarter,MSFT,AMZN,GOOG,NVDA,META,SMCI,TSMC-Revenue
0,2018-1,2993.0,2727.0,7269.0,3207.0,2812.0,835.0,NaN
1,2018-2,3980.0,2949.0,5507.0,3123.0,3460.0,981.0,NaN
2,2018-3,3602.0,2537.0,5282.0,3181.0,3342.0,971.0,NaN
3,2018-4,3707.0,3120.0,7081.0,2205.0,4301.0,931.0,NaN
4,2019-1,2565.0,2721.0,4638.0,2220.0,3837.0,743.0,NaN
5,2019-2,4051.0,2643.0,6126.0,2579.0,3633.0,854.0,NaN
6,2019-3,3385.0,3385.0,-10764.0,3014.0,3532.0,799.0,26.35
7,2019-4,3545.0,3940.0,23548.0,3105.0,4100.0,870.0,23.71
8,2020-1,3767.0,5428.0,6005.0,3080.0,3558.0,772.0,15.81
9,2020-2,4744.0,6614.0,5391.0,3866.0,3255.0,896.0,18.45


In [102]:
import yfinance as yf
msft = yf.Ticker("msft").quarterly_cashflow.T
print(msft.columns)
msft = msft[["Net PPE Purchase And Sale"]]/-1000000
msft = msft.reset_index().rename(columns={"index":"year-quarter"})
msft

Index(['Free Cash Flow', 'Repurchase Of Capital Stock', 'Repayment Of Debt',
       'Issuance Of Debt', 'Issuance Of Capital Stock', 'Capital Expenditure',
       'End Cash Position', 'Beginning Cash Position',
       'Effect Of Exchange Rate Changes', 'Changes In Cash',
       'Financing Cash Flow', 'Cash Flow From Continuing Financing Activities',
       'Net Other Financing Charges', 'Cash Dividends Paid',
       'Common Stock Dividend Paid', 'Net Common Stock Issuance',
       'Common Stock Payments', 'Common Stock Issuance',
       'Net Issuance Payments Of Debt', 'Net Short Term Debt Issuance',
       'Short Term Debt Issuance', 'Net Long Term Debt Issuance',
       'Long Term Debt Payments', 'Long Term Debt Issuance',
       'Investing Cash Flow', 'Cash Flow From Continuing Investing Activities',
       'Net Other Investing Changes', 'Net Investment Purchase And Sale',
       'Sale Of Investment', 'Purchase Of Investment',
       'Net Business Purchase And Sale', 'Purchase Of Bu

,year-quarter,Net PPE Purchase And Sale
0,2024-06-30,13873.0
1,2024-03-31,10952.0
2,2023-12-31,9735.0
3,2023-09-30,9917.0
4,2023-06-30,8943.0
5,2023-03-31,NaN


In [103]:
# transform year-quarter datetime to year-quarter format
msft['year-quarter'] = pd.to_datetime(msft['year-quarter'])
msft['year-quarter'] = msft['year-quarter'].dt.to_period('Q')
msft

,year-quarter,Net PPE Purchase And Sale
0,2024Q2,13873.0
1,2024Q1,10952.0
2,2023Q4,9735.0
3,2023Q3,9917.0
4,2023Q2,8943.0
5,2023Q1,NaN


In [104]:
# replace Q in year-quarter with -
msft['year-quarter'] = msft['year-quarter'].astype(str).str.replace('Q','-')
msft


,year-quarter,Net PPE Purchase And Sale
0,2024-2,13873.0
1,2024-1,10952.0
2,2023-4,9735.0
3,2023-3,9917.0
4,2023-2,8943.0
5,2023-1,NaN


In [105]:
merge_df = pd.merge(dataset, msft, how='outer', on='year-quarter')
merge_df["MSFT"].fillna(merge_df["Net PPE Purchase And Sale"], inplace=True)
merge_df = merge_df.drop(columns=["Net PPE Purchase And Sale"])
merge_df

,year-quarter,MSFT,AMZN,GOOG,NVDA,META,SMCI,TSMC-Revenue
0,2018-1,2993.0,2727.0,7269.0,3207.0,2812.0,835.0,NaN
1,2018-2,3980.0,2949.0,5507.0,3123.0,3460.0,981.0,NaN
2,2018-3,3602.0,2537.0,5282.0,3181.0,3342.0,971.0,NaN
3,2018-4,3707.0,3120.0,7081.0,2205.0,4301.0,931.0,NaN
4,2019-1,2565.0,2721.0,4638.0,2220.0,3837.0,743.0,NaN
5,2019-2,4051.0,2643.0,6126.0,2579.0,3633.0,854.0,NaN
6,2019-3,3385.0,3385.0,-10764.0,3014.0,3532.0,799.0,26.35
7,2019-4,3545.0,3940.0,23548.0,3105.0,4100.0,870.0,23.71
8,2020-1,3767.0,5428.0,6005.0,3080.0,3558.0,772.0,15.81
9,2020-2,4744.0,6614.0,5391.0,3866.0,3255.0,896.0,18.45


In [106]:
second_dataframe = yf.Ticker("amzn").quarterly_cashflow.T
second_dataframe = second_dataframe[["Net PPE Purchase And Sale"]]/-1000000
second_dataframe = second_dataframe.reset_index().rename(columns={"index":"year-quarter"})

second_dataframe['year-quarter'] = pd.to_datetime(msft['year-quarter'])
second_dataframe['year-quarter'] = second_dataframe['year-quarter'].dt.to_period('Q')
second_dataframe['year-quarter'] = second_dataframe['year-quarter'].astype(str).str.replace('Q','-')
second_dataframe

,year-quarter,Net PPE Purchase And Sale
0,2024-1,16393.0
1,2024-1,13935.0
2,2023-2,13353.0
3,2023-1,11298.0
4,2023-1,10412.0
5,2023-1,NaN


In [107]:
def fillemptyshit(df, stock: str):
    second_dataframe = yf.Ticker(f"{stock}").quarterly_cashflow.T
    second_dataframe = second_dataframe[["Net PPE Purchase And Sale"]]/-1000000
    second_dataframe = second_dataframe.reset_index().rename(columns={"index":"year-quarter"})

    second_dataframe['year-quarter'] = pd.to_datetime(second_dataframe['year-quarter'])
    second_dataframe['year-quarter'] = second_dataframe['year-quarter'].dt.to_period('Q')
    second_dataframe['year-quarter'] = second_dataframe['year-quarter'].astype(str).str.replace('Q','-')

    merge_df = pd.merge(df, second_dataframe, how='outer', on='year-quarter')

    stock = stock.upper()
    merge_df[f"{stock}"].fillna(merge_df["Net PPE Purchase And Sale"], inplace=True)
    merge_df = merge_df.drop(columns=["Net PPE Purchase And Sale"])
    return merge_df

merge_df = fillemptyshit(merge_df, "amzn")
merge_df = fillemptyshit(merge_df, "goog")
merge_df = fillemptyshit(merge_df, "meta")
merge_df

,year-quarter,MSFT,AMZN,GOOG,NVDA,META,SMCI,TSMC-Revenue
0,2018-1,2993.0,2727.0,7269.0,3207.0,2812.0,835.0,NaN
1,2018-2,3980.0,2949.0,5507.0,3123.0,3460.0,981.0,NaN
2,2018-3,3602.0,2537.0,5282.0,3181.0,3342.0,971.0,NaN
3,2018-4,3707.0,3120.0,7081.0,2205.0,4301.0,931.0,NaN
4,2019-1,2565.0,2721.0,4638.0,2220.0,3837.0,743.0,NaN
5,2019-2,4051.0,2643.0,6126.0,2579.0,3633.0,854.0,NaN
6,2019-3,3385.0,3385.0,-10764.0,3014.0,3532.0,799.0,26.35
7,2019-4,3545.0,3940.0,23548.0,3105.0,4100.0,870.0,23.71
8,2020-1,3767.0,5428.0,6005.0,3080.0,3558.0,772.0,15.81
9,2020-2,4744.0,6614.0,5391.0,3866.0,3255.0,896.0,18.45


In [108]:
def fill_nan_with_average(col):
    non_nan_values = col.dropna()[:4]  # Get the first 4 non-NaN values
    if len(non_nan_values) == 0:
        return col  # If there are no non-NaN values, return the column as it is
    average_value = non_nan_values.mean()  # Calculate the average
    return col.fillna(average_value)  # Fill NaN values with the calculated average

# Apply the function to each column
merge_df["TSMC-Revenue"] = fill_nan_with_average(merge_df["TSMC-Revenue"])
merge_df

,year-quarter,MSFT,AMZN,GOOG,NVDA,META,SMCI,TSMC-Revenue
0,2018-1,2993.0,2727.0,7269.0,3207.0,2812.0,835.0,21.08
1,2018-2,3980.0,2949.0,5507.0,3123.0,3460.0,981.0,21.08
2,2018-3,3602.0,2537.0,5282.0,3181.0,3342.0,971.0,21.08
3,2018-4,3707.0,3120.0,7081.0,2205.0,4301.0,931.0,21.08
4,2019-1,2565.0,2721.0,4638.0,2220.0,3837.0,743.0,21.08
5,2019-2,4051.0,2643.0,6126.0,2579.0,3633.0,854.0,21.08
6,2019-3,3385.0,3385.0,-10764.0,3014.0,3532.0,799.0,26.35
7,2019-4,3545.0,3940.0,23548.0,3105.0,4100.0,870.0,23.71
8,2020-1,3767.0,5428.0,6005.0,3080.0,3558.0,772.0,15.81
9,2020-2,4744.0,6614.0,5391.0,3866.0,3255.0,896.0,18.45


In [109]:
def fill_last_quarter(df, stock: str):
    second_dataframe = yf.Ticker(f"{stock}").quarterly_cashflow.T
    second_dataframe = second_dataframe[["Net Income From Continuing Operations"]]/-1000000
    second_dataframe = second_dataframe.reset_index().rename(columns={"index":"year-quarter"})
    print(second_dataframe)

    second_dataframe['year-quarter'] = pd.to_datetime(second_dataframe['year-quarter'])
    second_dataframe['year-quarter'] = second_dataframe['year-quarter'].dt.to_period('Q')
    second_dataframe['year-quarter'] = second_dataframe['year-quarter'].astype(str).str.replace('Q','-')

    merge_df = pd.merge(df, second_dataframe, how='outer', on='year-quarter')

    stock = stock.upper()
    merge_df[f"{stock}"].fillna(merge_df["Net Income From Continuing Operations"], inplace=True)
    merge_df = merge_df.drop(columns=["Net Income From Continuing Operations"])
    return merge_df

merge_df = fill_last_quarter(merge_df, "smci")

  year-quarter Net Income From Continuing Operations
0   2024-03-31                              -402.459
1   2023-12-31                              -295.968
2   2023-09-30                              -156.995
3   2023-06-30                              -193.569
4   2023-03-31                               -85.846
5   2022-12-31                                   NaN


In [110]:
merge_df["SMCI"].fillna(5310.0, inplace=True)
merge_df

,year-quarter,MSFT,AMZN,GOOG,NVDA,META,SMCI,TSMC-Revenue
0,2018-1,2993.0,2727.0,7269.0,3207.0,2812.0,835.0,21.08
1,2018-2,3980.0,2949.0,5507.0,3123.0,3460.0,981.0,21.08
2,2018-3,3602.0,2537.0,5282.0,3181.0,3342.0,971.0,21.08
3,2018-4,3707.0,3120.0,7081.0,2205.0,4301.0,931.0,21.08
4,2019-1,2565.0,2721.0,4638.0,2220.0,3837.0,743.0,21.08
5,2019-2,4051.0,2643.0,6126.0,2579.0,3633.0,854.0,21.08
6,2019-3,3385.0,3385.0,-10764.0,3014.0,3532.0,799.0,26.35
7,2019-4,3545.0,3940.0,23548.0,3105.0,4100.0,870.0,23.71
8,2020-1,3767.0,5428.0,6005.0,3080.0,3558.0,772.0,15.81
9,2020-2,4744.0,6614.0,5391.0,3866.0,3255.0,896.0,18.45


In [111]:
from sklearn.linear_model import LinearRegression

test_set_row_len = 1
max_row_number = len(merge_df)
test_set_row_number = max_row_number - 1
print(f'Test set row number: {test_set_row_number}')
print(f'Max row number: {max_row_number}')
pred_row_number = test_set_row_number
max_train_row_number = test_set_row_number-test_set_row_len
print(f'Max train row number: {max_train_row_number}')
print(f'Pred row number: {pred_row_number}')

X = merge_df.drop(columns=["NVDA", "year-quarter"])

X_train = X.iloc[:max_train_row_number]
y_train = merge_df["NVDA"].iloc[:max_train_row_number]

X_test = X.iloc[max_train_row_number:test_set_row_number]
y_test = merge_df["NVDA"].iloc[max_train_row_number:test_set_row_number]

X_pred = X.iloc[pred_row_number:]

model = LinearRegression()
model.fit(X_train, y_train)

y_pred_test = model.predict(X_test)

print(f'Predicted (Row 24): {y_pred_test}, Actual: {y_test.to_numpy()}')

y_pred_next = model.predict(X_pred)[0]


print(f'Predicted NVIDIA Revenue for Row 25: {y_pred_next}')

# Predicted (Row 24): 24875.60125209669, Actual: 26044.0


Test set row number: 25
Max row number: 26
Max train row number: 24
Pred row number: 25
Predicted (Row 24): [24875.6012521], Actual: [26044.]
Predicted NVIDIA Revenue for Row 25: 36988.73251032044


In [112]:
import numpy as np
absolute_error = np.mean(abs(y_test - y_pred_test))
absolute_error

np.float64(1168.3987479033094)